# Transformer (NMT)

The information and code is from https://pytorch.org/hub/pytorch_fairseq_translation/.

### Model Description

The Transformer, introduced in the paper Attention Is All You Need, is a powerful sequence-to-sequence modeling architecture capable of producing state-of-the-art neural machine translation (NMT) systems.

### Requirements

In [2]:
pip install bitarray fastBPE hydra-core omegaconf regex requests sacremoses subword_nmt scikit-learn sacrebleu

  Using cached bitarray-2.7.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (270 kB)
  Using cached fastBPE-0.1.0.tar.gz (35 kB)
  Preparing metadata (setup.py) ... done
  Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached sacremoses-0.0.53.tar.gz (880 kB)
  Preparing metadata (setup.py) ... done
  Using cached subword_nmt-0.3.8-py3-none-any.whl (27 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 89.3 kB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 83.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 108.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 106.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.3 kB/s eta 0:00:0000:0100:03m
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp39-cp39-li

In [1]:
import os
os.environ['HTTP_PROXY'] = 'http://proxy.vmware.com:3128'
os.environ['HTTPS_PROXY'] = 'http://proxy.vmware.com:3128'

### English-to-French Translation

In [2]:
import torch

# Load an En-Fr Transformer model trained on WMT'14 data :
en2fr = torch.hub.load('pytorch/fairseq', 'transformer.wmt14.en-fr', tokenizer='moses', bpe='subword_nmt')

# Use the GPU (optional):
en2fr.cuda()

# Translate with beam search:
fr = en2fr.translate('Hello world!', beam=5)
assert fr == 'Bonjour à tous !'

# Manually tokenize:
en_toks = en2fr.tokenize('Hello world!')
assert en_toks == 'Hello world !'

# Manually apply BPE:
en_bpe = en2fr.apply_bpe(en_toks)
assert en_bpe == 'H@@ ello world !'

# Manually binarize:
en_bin = en2fr.binarize(en_bpe)
assert en_bin.tolist() == [329, 14044, 682, 812, 2]

# Generate five translations with top-k sampling:
fr_bin = en2fr.generate(en_bin, beam=5, sampling=True, sampling_topk=20)
assert len(fr_bin) == 5

# Convert one of the samples to a string and detokenize
fr_sample = fr_bin[0]['tokens']
fr_bpe = en2fr.string(fr_sample)
fr_toks = en2fr.remove_bpe(fr_bpe)
fr = en2fr.detokenize(fr_toks)
assert fr == en2fr.decode(fr_sample)

Downloading: "https://github.com/pytorch/fairseq/zipball/main" to /home/jovyan/.cache/torch/hub/main.zip
2023-06-30 09:33:05.380475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 09:33:06.193495: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:root:running build_ext
/home/venv/lib/python3.9/site-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
INFO:root:cythoning fairseq/data/data_utils_fast.pyx to fair

In [3]:
print(fr)

Bonjour à tous !


### English-to-German Translation

In [4]:
import torch

# Load an En-De Transformer model trained on WMT'19 data:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')

# Access the underlying TransformerModel
assert isinstance(en2de.models[0], torch.nn.Module)

# Translate from En-De
de = en2de.translate('PyTorch Hub is a pre-trained model repository designed to facilitate research reproducibility.')
assert de == 'PyTorch Hub ist ein vorgefertigtes Modell-Repository, das die Reproduzierbarkeit der Forschung erleichtern soll.'

Using cache found in /home/jovyan/.cache/torch/hub/pytorch_fairseq_main
INFO:fairseq.file_utils:https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz not found in cache, downloading to /home/model-server/tmp/tmpg2nwswgq
100%|██████████| 2988854245/2988854245 [02:37<00:00, 18933814.19B/s]
INFO:fairseq.file_utils:copying /home/model-server/tmp/tmpg2nwswgq to cache at /home/jovyan/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
INFO:fairseq.file_utils:creating metadata file for /home/jovyan/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
INFO:fairseq.file_utils:removing temp file /home/model-server/tmp/tmpg2nwswgq
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_m

In [5]:
print(de)

PyTorch Hub ist ein vorgefertigtes Modell-Repository, das die Reproduzierbarkeit der Forschung erleichtern soll.


### A round-trip translation to create a paraphrase

In [6]:
# Round-trip translations between English and German:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

paraphrase = de2en.translate(en2de.translate('PyTorch Hub is an awesome interface!'))
assert paraphrase == 'PyTorch Hub is a fantastic interface!'

# Compare the results with English-Russian round-trip translation:
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')

paraphrase = ru2en.translate(en2ru.translate('PyTorch Hub is an awesome interface!'))
assert paraphrase == 'PyTorch is a great interface!'

Using cache found in /home/jovyan/.cache/torch/hub/pytorch_fairseq_main
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz from cache at /home/jovyan/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
INFO:fairseq.tasks.translation:[en] dictionary: 42024 types
INFO:fairseq.tasks.translation:[de] dictionary: 42024 types
INFO:fairseq.models.fairseq_model:{'_name': None, 'common': {'_name': None, 'no_progress_bar': True, 'log_interval': 100, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': '', 'wandb_project': None, 'azureml_logging': False, 'seed': 2, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window':

In [7]:
print(paraphrase)

PyTorch is a great interface!
